# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','Speed.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

Plot the speed definition graphic

In [ ]:
def plot_speed_def(ax):
    x0=np.arange(0,7,.01)
    y0=10*x0+10

    x1=np.arange(7, 8.16,.01)
    y1=-60*x1+500

    x=np.append(x0,x1)
    y=np.append(y0,y1)

    y=smooth(y*1.1,sigma=50,mode='constant')

    x40=np.where(np.logical_and(y<=40,x>7))[0][0]
    x60=np.where(np.logical_and(y<=60,x>7))[0][0]
    #plot the traj
    ax.plot(x, y, lw=2, c='navy', )
    #plot the 40 and 60 horizontal lines
    ax.hlines(y=40, xmin=-10, xmax=x[x40], lw=ax.spines['bottom'].get_lw(), ls='--')
    ax.hlines(y=60, xmin=-10, xmax=x[x60],lw=ax.spines['bottom'].get_lw(), ls='--')
    #plot the red piece
    ax.plot([x[x40],x[x60]],[40,60], lw=ax.spines['bottom'].get_lw(), c='r',zorder=4)
    #plot the x-axis at the red piece
    ax.arrow(x[x60],60,1.3,0,lw=ax.spines['bottom'].get_lw(),head_width=2,head_length=.2, ec='r', fc='r',zorder=5)
    # plot the curve
    x2=7.5
    y2=y[np.where(x>=x2)[0][0]]
    ax.add_patch(FancyArrowPatch(posA=(x[x60]+.8,62), posB=(x2-.1,y2-2),
                                 arrowstyle='-', connectionstyle='arc3,rad=-.5',color='r',
                                 mutation_scale=15,lw=ax.spines['bottom'].get_lw(),zorder=6))
    #writing 
    ax.text(7.9,40,'speed',rotation=-75, c='r', fontsize='xx-small',ha='left',va='center')
    
    
    ax.set_xticks([0,1,2,3,4,5,6,7,8])
    ax.set_yticks(range(0,91,10))
    ax.set_yticklabels([0,'',20,'',40,'',60,'',80,''])
    ax.spines['bottom'].set_bounds(0,8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_bounds(0,90)
    ax.set_xlim([-.1,9])
    ax.set_ylim([-2,90])
    ax.set_ylabel('Position (cm)')
    ax.set_xlabel('Trial time (s)')


In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    plot_speed_def(ax)

---

plot the speed correlation plot for LATE animals

In [ ]:
def late_lesion_correlation_with_size(root, ax, Profiles, badAnimals, color, TaskParamToPlot:str,
                                      preSlice=slice(-5,None), postSlice=slice(0,5)):
    
    _,sessionDict=event_detect(root, Profiles[0], Profiles[1], badAnimals=badAnimals)
    Results,nSessionPre,nSessionPost=event_statistic2(root,
                                                 sessionDict,
                                                 parameter=param,
                                                 redo=False,
                                                 TaskParamToPlot=[TaskParamToPlot])

    data=np.array(list(Results[TaskParamToPlot].values()))
    slicedPreData  = np.ones((data.shape))*np.nan
    slicedPostData = np.ones((data.shape))*np.nan
    
    for row,Dnan in enumerate(data):
        Dpre= Dnan[:nSessionPre][~ np.isnan(Dnan[:nSessionPre])]
        Dpost=Dnan[nSessionPre:][~ np.isnan(Dnan[nSessionPre:])]
        preData =Dpre[preSlice]
        postData=Dpost[postSlice]

        slicedPreData[row,:len(preData)]=preData
        slicedPostData[row,:len(postData)]=postData
    
    yPre =np.nanmean(slicedPreData ,axis=1)
    yPost=np.nanmean(slicedPostData,axis=1)

    size=[]
    behav=[]
    animals=[]
    for i,animal in enumerate(Results[TaskParamToPlot].keys()):
        if np.isnan(yPre[i]) or np.isnan(yPost[i]):
            logging.error(f'{animal}: {TaskParamToPlot} not defined!')
            continue
        try:
            Hist=HistologyExcel(NAS_PATH,animal)
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        
        behav.append(yPost[i]-yPre[i])
        size.append(Hist.lesion_size())
        animals.append(animal)
        
        #plotting
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=4, c=color[tag], label=tag,edgecolors='k',linewidths=.1);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'$\Delta$ {TaskParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout',
              'Control-NoTimeout-Control','Control-Sharp','IncReward-Late-Sharp',
              'Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':['0','10'],
             'Speed':'10',
             'Tag':['Late-Lesion_DLS','Late-Lesion_DMS','Late-Lesion_DS',
                    'Late-Lesion_DS-Sharp','Late-Lesion_DMS-Sharp']
             }

    Profiles=(profile1,profile2)
    TaskParamToPlot="Forward Running Speed"
    preSlice=slice(-5,None)
    postSlice=slice(3,8)
    
    color={'DS':'r','DMS':'g', 'DLS':'b', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    
    a=late_lesion_correlation_with_size(root, ax=ax, Profiles=Profiles, badAnimals=[],color=color, TaskParamToPlot=TaskParamToPlot,
                                      preSlice=preSlice, postSlice=postSlice)

---

plot the speed correlation plot for any animal

In [ ]:
def behav_lesion_size_correlation(root,ax,PerfParamToPlot,AnimalProfile, color,
                                  animalList=None, Ndays=slice(-5,None), **kwargs):

    if animalList is None:
        animalList=batch_get_animal_list(root,AnimalProfile)

    size=[]
    behav=[]
    animals=[]
    for animal in animalList:
        try:
            i=HistologyExcel(NAS_PATH,animal)
        except AssertionError as e:
            logging.warning(repr(e))
            continue
        except Exception as e:
            logging.error(f'{animal}: {repr(e)}')
            continue
        

        res=animal_learning_stats(root, animal, PerfParamToPlot, profile=AnimalProfile,
                                  sessionSlice=Ndays, redo=False)
        
        try:
            t1=np.nanmedian(res)
            t3=i.lesion_size()
        except Exception as e:
            logging.warning(repr(e))
            continue
        behav.append(t1)
        size.append(t3)
        animals.append(animal)
    
        #PLOTTING
        _,tag=lesion_type(root,animal)
        ax.scatter(size[-1], behav[-1], s=4, c=color[tag], label=tag, **kwargs);
#         plt.annotate(animal[-3:], (size[-1],behav[-1]), fontsize='xx-small')
    
    ax.set_xticks(np.arange(0,1.01,.1))
    ax.set_xticklabels(['0','','','','','0.5','','','','','1'])
    ax.spines['bottom'].set_bounds(0,1)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-.02,1.02])
    ax.set_ylabel(f'{PerfParamToPlot}')
    ax.set_xlabel('Lesion size')

    
    return behav, size, animals

def add_legend_for_lesion_types(ax):
    r_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markerfacecolor='xkcd:gray', marker='o',markeredgewidth=.1,
                                       markeredgecolor='xkcd:black', markersize=3, label='Late')
    b_marker = matplotlib.lines.Line2D([], [], color=[0,0,0,0],
                                       markeredgecolor='xkcd:black', marker='D',markeredgewidth=.1,
                                       markerfacecolor='xkcd:gray', markersize=3, label='Early')

    leg=ax.legend(handles=[r_marker,b_marker],loc=(0,-.02),mode=None,ncol=3,
                  facecolor=None,edgecolor=None, fontsize=5,frameon=False,handletextpad=0,columnspacing=1)
    return leg

In [ ]:
if "__file__" not in dir():
    
    #Define desired profile dictionary
    AnimalProfile={'Type':'Good',
         'option': ['not used'],
         'rewardType':'Progressive',
         'initialSpeed':['10'],
         'Speed':'10',
         'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS']
                  }
    PerfParamToPlot="Forward Running Speed"
    color={'DS':'m','DMS':'b', 'DLS':'r', 'Control':'k'}
    
    plt.close('all')
    ax=plt.subplot(111)

    behav, size, animals=behav_lesion_size_correlation(root,ax,PerfParamToPlot,AnimalProfile,color, marker='v')
    
    r,p=stats.pearsonr(size, behav)
    print(r,p)

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
    NAS_PATH='/NAS02'


    
    #===============================================
    
    # GRID 1 PARAMS
    
    _s=.1
    ShiftDLS1 =-1*_s
    ShiftDMS1 =0*_s
    ShiftDS1  =1*_s
    
    nPre1=5
    nPost1=10
    
    TaskParamToPlot1="Forward Running Speed"
    
    profilePreLesion1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                     'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profileDLS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DLS']
                 }
    profileDMS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DMS','Late-Lesion_DMS-Sharp'],
                 }  
    profileDS1={'Type':'Good',
              'rewardType':'Progressive',
              'option':['not used', 'AsymmetricLesion'],
              'initialSpeed':['10','0'],
              'Speed':'10',
              'Tag':['Late-Lesion_DS','Late-Lesion_DS-Sharp']
                 }  

    ProfilesDLS1=(profilePreLesion1,profileDLS1,)
    ProfilesDMS1=(profilePreLesion1,profileDMS1,)
    ProfilesDS1 =(profilePreLesion1,profileDS1,)
    
    
    #================================================
    
    # GRID 3 PARAMS

    profileLesions3={'Type':'Good',
                     'rewardType':'Progressive',
                     'option':['not used', 'AsymmetricLesion'],
                     'initialSpeed':['0','10'],
                     'Speed':'10',
                     'Tag': list(set((*profileDLS1['Tag'], *profileDMS1['Tag'], *profileDS1['Tag'])))
                     }

    Profiles3=(profilePreLesion1,profileLesions3)
    
    preSlice3=slice(-3,None)
    postSlice3=slice(0,3)
    finSlice3=slice(7,10)
    
    
    #================================================
    
    # GRID 4 PARAMS

    profile4={'Type':'Good',
             'option': ['not used', 'AsymmetricLesion'],
             'rewardType':'Progressive',
             'initialSpeed':['10'],
             'Speed':'10',
             'Tag':['Early-Lesion_DS','Early-Lesion_DMS','Early-Lesion_DLS']}
    


Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(4,1.1)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        
    ##########################################
    # 3: % speed correlation
    gs3= fig.add_gridspec(nrows=1, ncols=1, left=0, bottom=0, right=.3, top=1)
    ax3= fig.add_subplot(gs3[0])
  
    behav3, size3, animals3=late_lesion_correlation_with_size(root, ax=ax3, Profiles=Profiles3, badAnimals=[],
                                                              color=ColorCode,
                                                              TaskParamToPlot=TaskParamToPlot1,
                                                              preSlice=preSlice3, postSlice=postSlice3)
    
    r3,p3=stats.pearsonr(size3, behav3)
    s=f'$n={len(animals3)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r3)+'\n'+'$p=$'+'{}'.format(SciNote(p3))
    ax3.text(1,10,s, ha='right',va='top', fontsize='xx-small')
    
    ax3.set_ylim([-52,12])
    ax3.spines['left'].set_bounds(-50,10)
    ax3.set_yticks([-50,-40,-30,-20,-10,0,10])
    ax3.set_yticklabels([-50,'',-30,'',-10,'',10])
    ax3.set_ylabel('$\Delta$Speed (cm/s)')
    
    
    
    
    ##########################################
    # 4: % All rats corrlation
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=.4, bottom=0, right=.7, top=1)
    ax4= fig.add_subplot(gs4[0])
    
    behav4, size4, animals4   =behav_lesion_size_correlation(root,ax4,TaskParamToPlot1,profile4,ColorCode,
                                                             marker='D',edgecolors='k',linewidths=.1,alpha=.8)
    behav4_, size4_, animals4_=behav_lesion_size_correlation(root,ax4,TaskParamToPlot1,Profiles3[1],ColorCode,
                                                             marker='o',edgecolors='k',linewidths=.1,alpha=.8)
    
    r4,p4=stats.pearsonr(size4+size4_, behav4+behav4_)
    s=f'$n={len(animals4+animals4_)}$ rats'+'\n'+'$r=$'+'$%+.2f$'%(r4)+'\n'+'$p=$'+'{}'.format(SciNote(p4))
    ax4.text(1,80,s, ha='right',va='top', fontsize=5)
    
    add_legend_for_lesion_types(ax4)
    
    ax4.set_ylim([20,85])
    ax4.set_ylabel('Speed (cm/s)')
    ax4.spines['left'].set_bounds(20,80)

    
    
    
    
    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(axes4[0],ax1,ax2,ax5,ax7,ax8,ax6)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.01]*len(AXES))
#     OFFX[5]=0.03
#     OFFX[[0,1,2,4,6]]=0.05
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Speed.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()